In [56]:
import pandas as pd

# ЗАГРУЖАЕМ

In [234]:
#Выгружаем все данные
df_user = pd.read_sql(
    """
    SELECT *
    FROM "user_data" 
    """,

    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)
df_post = pd.read_sql(
    """
    SELECT *
    FROM "post_text_df" 
    """,

    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_feed = pd.read_sql(
    """
    SELECT *
    FROM "feed_data" 
    LIMIT 500000
    
    """,

    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

# EDA + DATASET TRANSFORMATIONS

In [235]:
df_user.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [236]:
df_user.nunique()

user_id      163205
gender            2
age              76
country          11
city           3915
exp_group         5
os                2
source            2
dtype: int64

In [237]:
for col in ['country', 'exp_group', 'os', 'source']:
    ohe = pd.get_dummies(df_user[col], prefix=col, drop_first=True)
    df_user = pd.concat((df_user.drop(col, axis=1), ohe), axis=1)
    
df_user = df_user.drop('city', axis=1)
df_user

,user_id,gender,age,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,country_Switzerland,country_Turkey,country_Ukraine,exp_group_1,exp_group_2,exp_group_3,exp_group_4,os_iOS,source_organic
0,200,1,34,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,201,0,37,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,202,1,17,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,203,0,18,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
4,204,0,36,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,36,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
163201,168549,0,18,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1
163202,168550,1,41,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
163203,168551,0,38,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1


In [61]:
df_post.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


# NN test (Начало)

In [163]:
### Сделаем эмбеддинги постов с помощью моделей из 10 занятия

from transformers import AutoTokenizer
from transformers import BertModel  # https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
from transformers import RobertaModel  # https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaModel
from transformers import DistilBertModel  # https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertModel


def get_model(model_name):
    assert model_name in ['bert', 'roberta', 'distilbert']

    checkpoint_names = {
        'bert': 'bert-base-cased',  # https://huggingface.co/bert-base-cased
        'roberta': 'roberta-base',  # https://huggingface.co/roberta-base
        'distilbert': 'distilbert-base-cased'  # https://huggingface.co/distilbert-base-cased
    }

    model_classes = {
        'bert': BertModel,
        'roberta': RobertaModel,
        'distilbert': DistilBertModel
    }

    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(checkpoint_names[model_name])

In [164]:
tokenizer, model = get_model('distilbert')

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [166]:
### Сделаем датасет для постов

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding


class PostDataset(Dataset):
    def __init__(self, texts, tokenizer):
        super().__init__()

        self.texts = tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            return_token_type_ids=False,
            return_tensors='pt',
            truncation=True,
            padding=True
        )
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        return {'input_ids': self.texts['input_ids'][idx], 'attention_mask': self.texts['attention_mask'][idx]}

    def __len__(self):
        return len(self.texts['input_ids'])
    
    
dataset = PostDataset(df_post['text'].values.tolist(), tokenizer)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loader = DataLoader(dataset, batch_size=32, collate_fn=data_collator, pin_memory=True, shuffle=False)

b = next(iter(loader))

b

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  101,  1993,  4190,  ...,     0,     0,     0],
        [  101, 15386,  1116,  ...,  1300,  1107,   102],
        [  101,  3141,   186,  ..., 14099,  8478,   102],
        ...,
        [  101, 16972, 20647,  ...,     0,     0,     0],
        [  101,   137,   188,  ...,     0,     0,     0],
        [  101,   144, 22731,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [167]:
import torch
from tqdm import tqdm


@torch.inference_mode()
def get_embeddings_labels(model, loader):
    model.eval()
    
    total_embeddings = []
    
    for batch in tqdm(loader):
        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}

        embeddings = model(**batch)['last_hidden_state'][:, 0, :]

        total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0)

In [168]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)
#print(torch.cuda.get_device_name())

model = model.to(device)

cpu


In [169]:
embeddings = get_embeddings_labels(model, loader).numpy()

embeddings

100%|████████████████████████████████████████████████████████████████████████████████| 220/220 [35:43<00:00,  9.74s/it]


array([[ 3.63150805e-01,  4.89374883e-02, -2.64081478e-01, ...,
        -1.41593143e-01,  1.59181543e-02,  9.18847509e-05],
       [ 2.36416280e-01, -1.59501165e-01, -3.27798426e-01, ...,
        -2.89936155e-01,  1.19365126e-01, -1.62371807e-03],
       [ 3.75191540e-01, -1.13943890e-01, -2.40547210e-01, ...,
        -3.38919073e-01,  5.86935952e-02, -2.12656353e-02],
       ...,
       [ 3.40382457e-01,  6.64923489e-02, -1.63184613e-01, ...,
        -8.65627229e-02,  2.03403696e-01,  3.20905857e-02],
       [ 4.32092071e-01,  1.10914111e-02, -1.17306188e-01, ...,
         7.54014552e-02,  1.02739751e-01,  1.52741857e-02],
       [ 3.04277599e-01, -7.62156844e-02, -6.77586421e-02, ...,
        -5.43488227e-02,  2.44383588e-01, -1.41483117e-02]], dtype=float32)

In [171]:
### Пытаемся кластеризовать тексты

from sklearn.decomposition import PCA

centered = embeddings - embeddings.mean()

pca = PCA(n_components=20)
pca_decomp = pca.fit_transform(centered)

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=15, random_state=0).fit(pca_decomp)

df_post['TextCluster'] = kmeans.labels_

dists_columns = ['DistanceTo1thCluster',
                 'DistanceTo2thCluster',
                 'DistanceTo3thCluster',
                 'DistanceTo4thCluster',
                 'DistanceTo5thCluster',
                 'DistanceTo6thCluster',
                 'DistanceTo7thCluster',
                 'DistanceTo8thCluster',
                 'DistanceTo9thCluster',
                 'DistanceTo10thCluster',
                 'DistanceTo11thCluster',
                 'DistanceTo12thCluster',
                 'DistanceTo13thCluster',
                 'DistanceTo14thCluster',
                 'DistanceTo15thCluster']

dists_df = pd.DataFrame(
    data=kmeans.transform(pca_decomp),
    columns=dists_columns
)

dists_df.head()

,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,3.317282,3.626825,3.504385,2.107590,2.673781,3.362377,1.969380,3.350282,1.552706,2.856973,3.283484,3.328180,3.299252,3.274009,3.226359
1,3.175854,3.332851,3.201369,2.080955,2.344407,3.098747,1.947727,2.963042,1.431155,2.653073,3.028255,3.160940,2.774500,3.260491,3.178570
2,3.206180,3.420908,3.228868,2.183235,2.723449,3.286630,2.831849,2.970637,1.357996,2.913700,3.102815,3.294351,2.775532,3.399966,3.126604
3,3.616450,2.843541,3.649253,2.582405,3.227175,3.933870,3.203395,3.670003,2.122464,3.132410,3.520485,3.669621,3.559877,3.584817,3.369332
4,2.602314,3.101309,2.821453,1.537728,1.803597,3.012253,2.673844,2.581019,1.345078,2.419489,2.649331,2.551137,2.418449,2.591954,2.823423


In [172]:
df_post = pd.concat((df_post, dists_df), axis=1)

df_post.head()

,post_id,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8,3.317282,3.626825,3.504385,2.107590,2.673781,3.362377,1.969380,3.350282,1.552706,2.856973,3.283484,3.328180,3.299252,3.274009,3.226359
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,8,3.175854,3.332851,3.201369,2.080955,2.344407,3.098747,1.947727,2.963042,1.431155,2.653073,3.028255,3.160940,2.774500,3.260491,3.178570
2,3,Asian quake hits European shares\n\nShares in ...,business,8,3.206180,3.420908,3.228868,2.183235,2.723449,3.286630,2.831849,2.970637,1.357996,2.913700,3.102815,3.294351,2.775532,3.399966,3.126604
3,4,India power shares jump on debut\n\nShares in ...,business,8,3.616450,2.843541,3.649253,2.582405,3.227175,3.933870,3.203395,3.670003,2.122464,3.132410,3.520485,3.669621,3.559877,3.584817,3.369332
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,8,2.602314,3.101309,2.821453,1.537728,1.803597,3.012253,2.673844,2.581019,1.345078,2.419489,2.649331,2.551137,2.418449,2.591954,2.823423


# NN test (Конец)

# Новый признак (начало)

In [238]:
res = df_feed.groupby(
    df_feed['post_id']
).agg({'target': 'mean'})
res.head()

,target
post_id,
1,0.078125
2,0.107143
3,0.128571
4,0.140625
5,0.116667


In [240]:
res = df_feed[df_feed['target'] == 1]
pos = res.groupby(res['post_id']).agg({'target':'count'})

res = df_feed[df_feed['target'] == 0]
neg = res.groupby(res['post_id']).agg({'target':'count'})

In [175]:
post_test = df_post
post_test.head()

,post_id,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8,3.317282,3.626825,3.504385,2.107590,2.673781,3.362377,1.969380,3.350282,1.552706,2.856973,3.283484,3.328180,3.299252,3.274009,3.226359
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,8,3.175854,3.332851,3.201369,2.080955,2.344407,3.098747,1.947727,2.963042,1.431155,2.653073,3.028255,3.160940,2.774500,3.260491,3.178570
2,3,Asian quake hits European shares\n\nShares in ...,business,8,3.206180,3.420908,3.228868,2.183235,2.723449,3.286630,2.831849,2.970637,1.357996,2.913700,3.102815,3.294351,2.775532,3.399966,3.126604
3,4,India power shares jump on debut\n\nShares in ...,business,8,3.616450,2.843541,3.649253,2.582405,3.227175,3.933870,3.203395,3.670003,2.122464,3.132410,3.520485,3.669621,3.559877,3.584817,3.369332
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,8,2.602314,3.101309,2.821453,1.537728,1.803597,3.012253,2.673844,2.581019,1.345078,2.419489,2.649331,2.551137,2.418449,2.591954,2.823423


In [176]:
post_test = post_test.merge(pos, on='post_id', how='left').fillna(0)
post_test = post_test.merge(neg, on='post_id', how='left').fillna(0)
post_test

,post_id,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,...,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,target_x,target_y
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8,3.317282,3.626825,3.504385,2.107590,2.673781,3.362377,...,3.350282,1.552706,2.856973,3.283484,3.328180,3.299252,3.274009,3.226359,2.0,14.0
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,8,3.175854,3.332851,3.201369,2.080955,2.344407,3.098747,...,2.963042,1.431155,2.653073,3.028255,3.160940,2.774500,3.260491,3.178570,2.0,26.0
2,3,Asian quake hits European shares\n\nShares in ...,business,8,3.206180,3.420908,3.228868,2.183235,2.723449,3.286630,...,2.970637,1.357996,2.913700,3.102815,3.294351,2.775532,3.399966,3.126604,6.0,27.0
3,4,India power shares jump on debut\n\nShares in ...,business,8,3.616450,2.843541,3.649253,2.582405,3.227175,3.933870,...,3.670003,2.122464,3.132410,3.520485,3.669621,3.559877,3.584817,3.369332,5.0,21.0
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,8,2.602314,3.101309,2.821453,1.537728,1.803597,3.012253,...,2.581019,1.345078,2.419489,2.649331,2.551137,2.418449,2.591954,2.823423,3.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,0,0.879528,2.180948,2.797818,2.564929,2.130065,3.268162,...,1.614585,2.793543,2.632243,1.916686,1.534206,2.829822,1.416083,2.974245,8.0,39.0
7019,7316,I give this movie 2 stars purely because of it...,movie,0,0.698967,2.086715,2.490488,2.322584,2.111093,3.286735,...,1.692232,2.873902,2.375822,1.713674,1.606385,3.111119,1.003729,2.832663,1.0,8.0
7020,7317,I cant believe this film was allowed to be mad...,movie,0,1.161841,2.444997,2.193800,2.647159,2.271674,3.361959,...,1.826294,3.061269,2.378414,2.133450,1.821000,3.014389,1.761077,2.676703,1.0,20.0
7021,7318,The version I saw of this film was the Blockbu...,movie,13,1.423654,1.947482,3.242426,2.937189,2.261545,3.346242,...,1.566741,3.147256,3.061242,1.938101,1.043027,3.161317,1.028654,3.388523,2.0,19.0


In [177]:
post_test['mean_target'] = (post_test['target_x'] / post_test['target_y'])
mean_like = round(post_test['mean_target'].mean(), 3)
mean_like

0.115

In [178]:
post_test[post_test['mean_target'] >= mean_like]

,post_id,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,...,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,target_x,target_y,mean_target
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8,3.317282,3.626825,3.504385,2.107590,2.673781,3.362377,...,1.552706,2.856973,3.283484,3.328180,3.299252,3.274009,3.226359,2.0,14.0,0.142857
2,3,Asian quake hits European shares\n\nShares in ...,business,8,3.206180,3.420908,3.228868,2.183235,2.723449,3.286630,...,1.357996,2.913700,3.102815,3.294351,2.775532,3.399966,3.126604,6.0,27.0,0.222222
3,4,India power shares jump on debut\n\nShares in ...,business,8,3.616450,2.843541,3.649253,2.582405,3.227175,3.933870,...,2.122464,3.132410,3.520485,3.669621,3.559877,3.584817,3.369332,5.0,21.0,0.238095
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,8,2.602314,3.101309,2.821453,1.537728,1.803597,3.012253,...,1.345078,2.419489,2.649331,2.551137,2.418449,2.591954,2.823423,3.0,23.0,0.130435
6,14,Saudi investor picks up the Savoy\n\nLondons f...,business,8,2.994622,3.269539,3.188601,1.911811,1.809835,2.697434,...,1.497516,2.869183,2.922051,2.793576,2.648925,2.989812,3.312860,2.0,11.0,0.181818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7016,7313,"Oh God,what an idiotic movie!Incredibly cheap ...",movie,0,0.868447,2.199045,2.616758,2.775151,2.566123,3.451856,...,3.129943,2.555954,1.973326,1.604109,3.342596,1.365076,3.004099,4.0,15.0,0.266667
7017,7314,Although the likeliness of someone focusing on...,movie,0,1.179555,2.352704,2.846943,2.361339,2.511536,3.481241,...,2.955696,2.751148,1.662250,1.845379,3.259588,1.718896,3.117846,7.0,34.0,0.205882
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,0,0.879528,2.180948,2.797818,2.564929,2.130065,3.268162,...,2.793543,2.632243,1.916686,1.534206,2.829822,1.416083,2.974245,8.0,39.0,0.205128
7019,7316,I give this movie 2 stars purely because of it...,movie,0,0.698967,2.086715,2.490488,2.322584,2.111093,3.286735,...,2.873902,2.375822,1.713674,1.606385,3.111119,1.003729,2.832663,1.0,8.0,0.125000


In [179]:
import numpy as np
post_test['attraction'] = np.where(post_test['mean_target'] >= mean_like, True, False)
post_test

,post_id,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,...,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,target_x,target_y,mean_target,attraction
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8,3.317282,3.626825,3.504385,2.107590,2.673781,3.362377,...,2.856973,3.283484,3.328180,3.299252,3.274009,3.226359,2.0,14.0,0.142857,True
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,8,3.175854,3.332851,3.201369,2.080955,2.344407,3.098747,...,2.653073,3.028255,3.160940,2.774500,3.260491,3.178570,2.0,26.0,0.076923,False
2,3,Asian quake hits European shares\n\nShares in ...,business,8,3.206180,3.420908,3.228868,2.183235,2.723449,3.286630,...,2.913700,3.102815,3.294351,2.775532,3.399966,3.126604,6.0,27.0,0.222222,True
3,4,India power shares jump on debut\n\nShares in ...,business,8,3.616450,2.843541,3.649253,2.582405,3.227175,3.933870,...,3.132410,3.520485,3.669621,3.559877,3.584817,3.369332,5.0,21.0,0.238095,True
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,8,2.602314,3.101309,2.821453,1.537728,1.803597,3.012253,...,2.419489,2.649331,2.551137,2.418449,2.591954,2.823423,3.0,23.0,0.130435,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,0,0.879528,2.180948,2.797818,2.564929,2.130065,3.268162,...,2.632243,1.916686,1.534206,2.829822,1.416083,2.974245,8.0,39.0,0.205128,True
7019,7316,I give this movie 2 stars purely because of it...,movie,0,0.698967,2.086715,2.490488,2.322584,2.111093,3.286735,...,2.375822,1.713674,1.606385,3.111119,1.003729,2.832663,1.0,8.0,0.125000,True
7020,7317,I cant believe this film was allowed to be mad...,movie,0,1.161841,2.444997,2.193800,2.647159,2.271674,3.361959,...,2.378414,2.133450,1.821000,3.014389,1.761077,2.676703,1.0,20.0,0.050000,False
7021,7318,The version I saw of this film was the Blockbu...,movie,13,1.423654,1.947482,3.242426,2.937189,2.261545,3.346242,...,3.061242,1.938101,1.043027,3.161317,1.028654,3.388523,2.0,19.0,0.105263,False


In [180]:
post_test = post_test.drop(['target_x', 'target_y', 'mean_target'], axis =1)
post_test

,post_id,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,attraction
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8,3.317282,3.626825,3.504385,2.107590,2.673781,3.362377,1.969380,3.350282,1.552706,2.856973,3.283484,3.328180,3.299252,3.274009,3.226359,True
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,8,3.175854,3.332851,3.201369,2.080955,2.344407,3.098747,1.947727,2.963042,1.431155,2.653073,3.028255,3.160940,2.774500,3.260491,3.178570,False
2,3,Asian quake hits European shares\n\nShares in ...,business,8,3.206180,3.420908,3.228868,2.183235,2.723449,3.286630,2.831849,2.970637,1.357996,2.913700,3.102815,3.294351,2.775532,3.399966,3.126604,True
3,4,India power shares jump on debut\n\nShares in ...,business,8,3.616450,2.843541,3.649253,2.582405,3.227175,3.933870,3.203395,3.670003,2.122464,3.132410,3.520485,3.669621,3.559877,3.584817,3.369332,True
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,8,2.602314,3.101309,2.821453,1.537728,1.803597,3.012253,2.673844,2.581019,1.345078,2.419489,2.649331,2.551137,2.418449,2.591954,2.823423,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,0,0.879528,2.180948,2.797818,2.564929,2.130065,3.268162,3.195646,1.614585,2.793543,2.632243,1.916686,1.534206,2.829822,1.416083,2.974245,True
7019,7316,I give this movie 2 stars purely because of it...,movie,0,0.698967,2.086715,2.490488,2.322584,2.111093,3.286735,3.083744,1.692232,2.873902,2.375822,1.713674,1.606385,3.111119,1.003729,2.832663,True
7020,7317,I cant believe this film was allowed to be mad...,movie,0,1.161841,2.444997,2.193800,2.647159,2.271674,3.361959,3.267648,1.826294,3.061269,2.378414,2.133450,1.821000,3.014389,1.761077,2.676703,False
7021,7318,The version I saw of this film was the Blockbu...,movie,13,1.423654,1.947482,3.242426,2.937189,2.261545,3.346242,3.379467,1.566741,3.147256,3.061242,1.938101,1.043027,3.161317,1.028654,3.388523,False


In [181]:
post_test['attraction'] = post_test['attraction'].replace([True, False], [1,0])
post_test

,post_id,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,attraction
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8,3.317282,3.626825,3.504385,2.107590,2.673781,3.362377,1.969380,3.350282,1.552706,2.856973,3.283484,3.328180,3.299252,3.274009,3.226359,1
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,8,3.175854,3.332851,3.201369,2.080955,2.344407,3.098747,1.947727,2.963042,1.431155,2.653073,3.028255,3.160940,2.774500,3.260491,3.178570,0
2,3,Asian quake hits European shares\n\nShares in ...,business,8,3.206180,3.420908,3.228868,2.183235,2.723449,3.286630,2.831849,2.970637,1.357996,2.913700,3.102815,3.294351,2.775532,3.399966,3.126604,1
3,4,India power shares jump on debut\n\nShares in ...,business,8,3.616450,2.843541,3.649253,2.582405,3.227175,3.933870,3.203395,3.670003,2.122464,3.132410,3.520485,3.669621,3.559877,3.584817,3.369332,1
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,8,2.602314,3.101309,2.821453,1.537728,1.803597,3.012253,2.673844,2.581019,1.345078,2.419489,2.649331,2.551137,2.418449,2.591954,2.823423,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,0,0.879528,2.180948,2.797818,2.564929,2.130065,3.268162,3.195646,1.614585,2.793543,2.632243,1.916686,1.534206,2.829822,1.416083,2.974245,1
7019,7316,I give this movie 2 stars purely because of it...,movie,0,0.698967,2.086715,2.490488,2.322584,2.111093,3.286735,3.083744,1.692232,2.873902,2.375822,1.713674,1.606385,3.111119,1.003729,2.832663,1
7020,7317,I cant believe this film was allowed to be mad...,movie,0,1.161841,2.444997,2.193800,2.647159,2.271674,3.361959,3.267648,1.826294,3.061269,2.378414,2.133450,1.821000,3.014389,1.761077,2.676703,0
7021,7318,The version I saw of this film was the Blockbu...,movie,13,1.423654,1.947482,3.242426,2.937189,2.261545,3.346242,3.379467,1.566741,3.147256,3.061242,1.938101,1.043027,3.161317,1.028654,3.388523,0


# Новый признак (конец)

In [182]:
#ohe = pd.get_dummies(df_post['topic'], prefix='topic', drop_first=True)
#df_post = pd.concat((df_post.drop('topic', axis=1), ohe), axis=1)
#df_post

ohe = pd.get_dummies(post_test['topic'], prefix='topic', drop_first=True)
post_test = pd.concat((post_test.drop('topic', axis=1), ohe), axis=1)
post_test.head()

,post_id,text,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,...,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,attraction,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,UK economy facing major risks\n\nThe UK manufa...,8,3.317282,3.626825,3.504385,2.107590,2.673781,3.362377,1.969380,...,3.299252,3.274009,3.226359,1,0,0,0,0,0,0
1,2,Aids and climate top Davos agenda\n\nClimate c...,8,3.175854,3.332851,3.201369,2.080955,2.344407,3.098747,1.947727,...,2.774500,3.260491,3.178570,0,0,0,0,0,0,0
2,3,Asian quake hits European shares\n\nShares in ...,8,3.206180,3.420908,3.228868,2.183235,2.723449,3.286630,2.831849,...,2.775532,3.399966,3.126604,1,0,0,0,0,0,0
3,4,India power shares jump on debut\n\nShares in ...,8,3.616450,2.843541,3.649253,2.582405,3.227175,3.933870,3.203395,...,3.559877,3.584817,3.369332,1,0,0,0,0,0,0
4,5,Lacroix label bought by US firm\n\nLuxury good...,8,2.602314,3.101309,2.821453,1.537728,1.803597,3.012253,2.673844,...,2.418449,2.591954,2.823423,1,0,0,0,0,0,0


In [241]:
df_feed.head()

,timestamp,user_id,post_id,action,target
0,2021-10-13 11:32:51,33578,1187,view,0
1,2021-10-13 11:33:07,33578,5390,view,0
2,2021-10-13 11:35:53,33578,6724,view,0
3,2021-10-13 11:38:22,33578,1440,view,0
4,2021-10-13 11:40:43,33578,4071,view,0


In [184]:
#ohe = pd.get_dummies(df_feed['action'], prefix='action', drop_first=True)
#df_feed = pd.concat((df_feed.drop(['timestamp', 'action'], axis=1), ohe), axis=1)

In [242]:
df_feed = df_feed.drop(['timestamp', 'action'], axis=1)

In [188]:
df_feed

,user_id,post_id,target
0,101572,3460,0
1,101572,4955,0
2,101572,1832,0
3,101572,5614,0
4,101572,6916,0
...,...,...,...
199995,80579,72,0
199996,80579,622,0
199997,80579,2771,0
199998,80579,3561,0


In [243]:
df_user.corr()

,user_id,gender,age,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,country_Switzerland,country_Turkey,country_Ukraine,exp_group_1,exp_group_2,exp_group_3,exp_group_4,os_iOS,source_organic
user_id,1.000000,-0.004867,0.003124,0.000896,0.001865,0.001883,-0.002903,-0.000900,-0.001217,0.001761,0.002605,0.001166,-0.001858,0.002634,-0.002667,-0.004300,-0.000823,-0.003752,0.837170
gender,-0.004867,1.000000,0.000582,0.003896,0.004688,0.002935,-0.002949,-0.004446,-0.002064,-0.002720,0.002615,0.002816,0.002687,-0.001397,0.001753,-0.002799,-0.001275,-0.004656,-0.004548
age,0.003124,0.000582,1.000000,0.001147,0.002760,0.003068,-0.000616,0.000400,0.000070,-0.001825,0.002325,0.002499,0.001420,0.001361,-0.002649,0.001283,-0.000978,0.001518,0.002195
country_Belarus,0.000896,0.003896,0.001147,1.000000,-0.004634,-0.004742,-0.014274,-0.020203,-0.004702,-0.382141,-0.004523,-0.014306,-0.033160,-0.000059,-0.001532,-0.002193,0.002621,-0.000621,0.003121
country_Cyprus,0.001865,0.004688,0.002760,-0.004634,1.000000,-0.001067,-0.003212,-0.004546,-0.001058,-0.085991,-0.001018,-0.003219,-0.007462,0.000001,-0.000462,0.004205,0.000565,-0.001041,0.000752
country_Estonia,0.001883,0.002935,0.003068,-0.004742,-0.001067,1.000000,-0.003287,-0.004652,-0.001083,-0.087993,-0.001042,-0.003294,-0.007636,0.002044,-0.000265,0.000121,-0.006234,-0.000553,0.001495
country_Finland,-0.002903,-0.002949,-0.000616,-0.014274,-0.003212,-0.003287,1.000000,-0.014004,-0.003259,-0.264889,-0.003135,-0.009916,-0.022986,0.000969,0.002095,-0.003579,0.000772,-0.002443,-0.003048
country_Kazakhstan,-0.000900,-0.004446,0.000400,-0.020203,-0.004546,-0.004652,-0.014004,1.000000,-0.004613,-0.374913,-0.004438,-0.014035,-0.032533,0.002070,-0.003649,0.001455,0.000787,0.002199,-0.000612
country_Latvia,-0.001217,-0.002064,0.000070,-0.004702,-0.001058,-0.001083,-0.003259,-0.004613,1.000000,-0.087248,-0.001033,-0.003266,-0.007571,-0.002339,-0.000455,0.000871,0.000090,0.001817,0.000013
country_Russia,0.001761,-0.002720,-0.001825,-0.382141,-0.085991,-0.087993,-0.264889,-0.374913,-0.087248,1.000000,-0.083941,-0.265474,-0.615360,0.000681,0.002033,-0.000339,-0.002478,-0.000263,0.001347


In [244]:
#Проверяем типы данных в датафреймах
print(df_user.dtypes)
print()
print(df_post.dtypes)
print()
print(df_feed.dtypes)

user_id                int64
gender                 int64
age                    int64
country_Belarus        uint8
country_Cyprus         uint8
country_Estonia        uint8
country_Finland        uint8
country_Kazakhstan     uint8
country_Latvia         uint8
country_Russia         uint8
country_Switzerland    uint8
country_Turkey         uint8
country_Ukraine        uint8
exp_group_1            uint8
exp_group_2            uint8
exp_group_3            uint8
exp_group_4            uint8
os_iOS                 uint8
source_organic         uint8
dtype: object

post_id     int64
text       object
topic      object
dtype: object

user_id    int64
post_id    int64
target     int64
dtype: object


In [245]:
#Проверяем пропуски
#df_user.isna().sum() #нет пропусков
#df_post.isna().sum() #нет пропусков
df_feed.isna().sum() #нет пропусков

user_id    0
post_id    0
target     0
dtype: int64

In [246]:
#df_feed.groupby(['user_id', 'post_id'],as_index=False).agg({'target':'sum','action':'sum'})

In [247]:
post_test.head()

,post_id,text,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,...,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,attraction,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,UK economy facing major risks\n\nThe UK manufa...,8,3.317282,3.626825,3.504385,2.107590,2.673781,3.362377,1.969380,...,3.299252,3.274009,3.226359,1,0,0,0,0,0,0
1,2,Aids and climate top Davos agenda\n\nClimate c...,8,3.175854,3.332851,3.201369,2.080955,2.344407,3.098747,1.947727,...,2.774500,3.260491,3.178570,0,0,0,0,0,0,0
2,3,Asian quake hits European shares\n\nShares in ...,8,3.206180,3.420908,3.228868,2.183235,2.723449,3.286630,2.831849,...,2.775532,3.399966,3.126604,1,0,0,0,0,0,0
3,4,India power shares jump on debut\n\nShares in ...,8,3.616450,2.843541,3.649253,2.582405,3.227175,3.933870,3.203395,...,3.559877,3.584817,3.369332,1,0,0,0,0,0,0
4,5,Lacroix label bought by US firm\n\nLuxury good...,8,2.602314,3.101309,2.821453,1.537728,1.803597,3.012253,2.673844,...,2.418449,2.591954,2.823423,1,0,0,0,0,0,0


In [248]:
test = df_user.merge(df_feed, on = 'user_id', how = 'left').fillna(0)

#test = test[test['post_id'] != 'empty']

test = test.merge(post_test, on = 'post_id').fillna(0)
test = test.drop('text', axis=1)
test
#df_feed[df_feed['user_id']==120098].merge(df_post, on = 'post_id', how = 'left')

,user_id,gender,age,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,...,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,attraction,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,12421,1,34,0,0,0,0,0,0,1,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
1,12421,1,34,0,0,0,0,0,0,1,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
2,12439,0,27,0,0,0,0,0,0,1,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
3,12472,1,39,0,0,0,0,0,0,1,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
4,12472,1,39,0,0,0,0,0,0,1,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,167248,0,35,0,0,0,0,0,0,1,...,3.140694,1.048808,3.071775,1,0,0,1,0,0,0
499996,167261,0,15,0,0,0,0,0,0,1,...,3.140694,1.048808,3.071775,1,0,0,1,0,0,0
499997,167268,0,47,0,0,0,0,0,0,1,...,3.140694,1.048808,3.071775,1,0,0,1,0,0,0
499998,167268,0,47,0,0,0,0,0,0,1,...,3.140694,1.048808,3.071775,1,0,0,1,0,0,0


In [249]:
col_to_int = ['post_id', 'target', 'topic_covid', 'topic_entertainment',
               'topic_movie', 'topic_politics', 'topic_sport', 'topic_tech'] #'action_view' убран
test[col_to_int] = test[col_to_int].astype(int)
#test.nunique()
test

,user_id,gender,age,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,...,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,attraction,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,12421,1,34,0,0,0,0,0,0,1,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
1,12421,1,34,0,0,0,0,0,0,1,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
2,12439,0,27,0,0,0,0,0,0,1,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
3,12472,1,39,0,0,0,0,0,0,1,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
4,12472,1,39,0,0,0,0,0,0,1,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,167248,0,35,0,0,0,0,0,0,1,...,3.140694,1.048808,3.071775,1,0,0,1,0,0,0
499996,167261,0,15,0,0,0,0,0,0,1,...,3.140694,1.048808,3.071775,1,0,0,1,0,0,0
499997,167268,0,47,0,0,0,0,0,0,1,...,3.140694,1.048808,3.071775,1,0,0,1,0,0,0
499998,167268,0,47,0,0,0,0,0,0,1,...,3.140694,1.048808,3.071775,1,0,0,1,0,0,0


In [250]:
test = test.drop(['user_id', 'post_id'], axis=1)
test

,gender,age,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,country_Switzerland,...,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,attraction,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,34,0,0,0,0,0,0,1,0,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
1,1,34,0,0,0,0,0,0,1,0,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
2,0,27,0,0,0,0,0,0,1,0,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
3,1,39,0,0,0,0,0,0,1,0,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
4,1,39,0,0,0,0,0,0,1,0,...,2.954602,1.596257,3.105802,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,35,0,0,0,0,0,0,1,0,...,3.140694,1.048808,3.071775,1,0,0,1,0,0,0
499996,0,15,0,0,0,0,0,0,1,0,...,3.140694,1.048808,3.071775,1,0,0,1,0,0,0
499997,0,47,0,0,0,0,0,0,1,0,...,3.140694,1.048808,3.071775,1,0,0,1,0,0,0
499998,0,47,0,0,0,0,0,0,1,0,...,3.140694,1.048808,3.071775,1,0,0,1,0,0,0


# Тестируем работу конката при запросе

In [251]:
test_row = df_user.iloc[[0]]
test_row

,user_id,gender,age,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,country_Switzerland,country_Turkey,country_Ukraine,exp_group_1,exp_group_2,exp_group_3,exp_group_4,os_iOS,source_organic
0,200,1,34,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0


In [252]:
df_user[df_user['user_id'] == 200]

,user_id,gender,age,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,country_Switzerland,country_Turkey,country_Ukraine,exp_group_1,exp_group_2,exp_group_3,exp_group_4,os_iOS,source_organic
0,200,1,34,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0


In [253]:
#df_user(df_user['user_id'] == 200)
test_row = df_user.iloc[[0]].drop('user_id', axis=1)
test_row 

,gender,age,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,country_Switzerland,country_Turkey,country_Ukraine,exp_group_1,exp_group_2,exp_group_3,exp_group_4,os_iOS,source_organic
0,1,34,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0


In [254]:
all_posts = pd.concat((test_row, post_test.drop(['text'], axis=1)), axis=1).fillna(method='ffill')
all_posts

,gender,age,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,country_Switzerland,...,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,attraction,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.299252,3.274009,3.226359,1,0,0,0,0,0,0
1,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.774500,3.260491,3.178570,0,0,0,0,0,0,0
2,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.775532,3.399966,3.126604,1,0,0,0,0,0,0
3,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.559877,3.584817,3.369332,1,0,0,0,0,0,0
4,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.418449,2.591954,2.823423,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.829822,1.416083,2.974245,1,0,0,1,0,0,0
7019,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.111119,1.003729,2.832663,1,0,0,1,0,0,0
7020,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.014389,1.761077,2.676703,0,0,0,1,0,0,0
7021,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.161317,1.028654,3.388523,0,0,0,1,0,0,0


# СТРОИМ МОДЕЛЬ

In [255]:
from sklearn.model_selection import train_test_split

X = test.drop(['target'], axis=1)
y = test['target']

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=1)

In [256]:
#Вариант 2
from catboost import CatBoostClassifier

model = CatBoostClassifier(learning_rate=0.02)

model.fit(X_train, y_train)


0:	learn: 0.6767129	total: 54.8ms	remaining: 54.8s
1:	learn: 0.6611702	total: 85.8ms	remaining: 42.8s
2:	learn: 0.6462094	total: 115ms	remaining: 38.4s
3:	learn: 0.6319777	total: 146ms	remaining: 36.4s
4:	learn: 0.6185180	total: 176ms	remaining: 35s
5:	learn: 0.6056953	total: 205ms	remaining: 34s
6:	learn: 0.5934656	total: 237ms	remaining: 33.6s
7:	learn: 0.5817909	total: 267ms	remaining: 33.1s
8:	learn: 0.5707095	total: 299ms	remaining: 32.9s
9:	learn: 0.5601260	total: 331ms	remaining: 32.8s
10:	learn: 0.5500722	total: 362ms	remaining: 32.6s
11:	learn: 0.5404454	total: 392ms	remaining: 32.3s
12:	learn: 0.5312993	total: 423ms	remaining: 32.1s
13:	learn: 0.5226146	total: 453ms	remaining: 31.9s
14:	learn: 0.5142049	total: 488ms	remaining: 32.1s
15:	learn: 0.5062632	total: 520ms	remaining: 32s
16:	learn: 0.4987212	total: 553ms	remaining: 32s
17:	learn: 0.4915639	total: 582ms	remaining: 31.8s
18:	learn: 0.4845441	total: 611ms	remaining: 31.5s
19:	learn: 0.4778698	total: 641ms	remaining: 31

167:	learn: 0.3371009	total: 5.38s	remaining: 26.7s
168:	learn: 0.3370592	total: 5.42s	remaining: 26.6s
169:	learn: 0.3370409	total: 5.45s	remaining: 26.6s
170:	learn: 0.3370144	total: 5.48s	remaining: 26.6s
171:	learn: 0.3369888	total: 5.52s	remaining: 26.6s
172:	learn: 0.3369621	total: 5.55s	remaining: 26.5s
173:	learn: 0.3369267	total: 5.58s	remaining: 26.5s
174:	learn: 0.3369019	total: 5.63s	remaining: 26.5s
175:	learn: 0.3368721	total: 5.66s	remaining: 26.5s
176:	learn: 0.3368272	total: 5.69s	remaining: 26.5s
177:	learn: 0.3368075	total: 5.72s	remaining: 26.4s
178:	learn: 0.3367877	total: 5.75s	remaining: 26.4s
179:	learn: 0.3367605	total: 5.79s	remaining: 26.4s
180:	learn: 0.3367231	total: 5.82s	remaining: 26.4s
181:	learn: 0.3366881	total: 5.86s	remaining: 26.3s
182:	learn: 0.3366507	total: 5.89s	remaining: 26.3s
183:	learn: 0.3366310	total: 5.92s	remaining: 26.3s
184:	learn: 0.3366031	total: 5.95s	remaining: 26.2s
185:	learn: 0.3365783	total: 5.99s	remaining: 26.2s
186:	learn: 

327:	learn: 0.3347359	total: 10.6s	remaining: 21.6s
328:	learn: 0.3347288	total: 10.6s	remaining: 21.6s
329:	learn: 0.3347177	total: 10.6s	remaining: 21.6s
330:	learn: 0.3347142	total: 10.6s	remaining: 21.5s
331:	learn: 0.3346995	total: 10.7s	remaining: 21.5s
332:	learn: 0.3346900	total: 10.7s	remaining: 21.4s
333:	learn: 0.3346815	total: 10.7s	remaining: 21.4s
334:	learn: 0.3346731	total: 10.8s	remaining: 21.4s
335:	learn: 0.3346625	total: 10.8s	remaining: 21.3s
336:	learn: 0.3346573	total: 10.8s	remaining: 21.3s
337:	learn: 0.3346536	total: 10.9s	remaining: 21.3s
338:	learn: 0.3346458	total: 10.9s	remaining: 21.2s
339:	learn: 0.3346416	total: 10.9s	remaining: 21.2s
340:	learn: 0.3346364	total: 11s	remaining: 21.2s
341:	learn: 0.3346262	total: 11s	remaining: 21.2s
342:	learn: 0.3346167	total: 11s	remaining: 21.1s
343:	learn: 0.3346019	total: 11.1s	remaining: 21.1s
344:	learn: 0.3345970	total: 11.1s	remaining: 21.1s
345:	learn: 0.3345936	total: 11.1s	remaining: 21s
346:	learn: 0.334583

489:	learn: 0.3337311	total: 15.7s	remaining: 16.3s
490:	learn: 0.3337245	total: 15.7s	remaining: 16.3s
491:	learn: 0.3337190	total: 15.7s	remaining: 16.3s
492:	learn: 0.3337147	total: 15.8s	remaining: 16.2s
493:	learn: 0.3337093	total: 15.8s	remaining: 16.2s
494:	learn: 0.3337045	total: 15.8s	remaining: 16.2s
495:	learn: 0.3336971	total: 15.9s	remaining: 16.1s
496:	learn: 0.3336935	total: 15.9s	remaining: 16.1s
497:	learn: 0.3336854	total: 15.9s	remaining: 16.1s
498:	learn: 0.3336793	total: 16s	remaining: 16s
499:	learn: 0.3336760	total: 16s	remaining: 16s
500:	learn: 0.3336735	total: 16s	remaining: 16s
501:	learn: 0.3336656	total: 16.1s	remaining: 15.9s
502:	learn: 0.3336574	total: 16.1s	remaining: 15.9s
503:	learn: 0.3336522	total: 16.1s	remaining: 15.9s
504:	learn: 0.3336451	total: 16.2s	remaining: 15.8s
505:	learn: 0.3336374	total: 16.2s	remaining: 15.8s
506:	learn: 0.3336340	total: 16.2s	remaining: 15.8s
507:	learn: 0.3336310	total: 16.3s	remaining: 15.7s
508:	learn: 0.3336286	to

654:	learn: 0.3329133	total: 21.1s	remaining: 11.1s
655:	learn: 0.3329084	total: 21.2s	remaining: 11.1s
656:	learn: 0.3329035	total: 21.2s	remaining: 11.1s
657:	learn: 0.3328977	total: 21.2s	remaining: 11s
658:	learn: 0.3328931	total: 21.3s	remaining: 11s
659:	learn: 0.3328890	total: 21.3s	remaining: 11s
660:	learn: 0.3328858	total: 21.3s	remaining: 10.9s
661:	learn: 0.3328826	total: 21.4s	remaining: 10.9s
662:	learn: 0.3328786	total: 21.4s	remaining: 10.9s
663:	learn: 0.3328695	total: 21.4s	remaining: 10.8s
664:	learn: 0.3328668	total: 21.5s	remaining: 10.8s
665:	learn: 0.3328642	total: 21.5s	remaining: 10.8s
666:	learn: 0.3328576	total: 21.5s	remaining: 10.7s
667:	learn: 0.3328552	total: 21.6s	remaining: 10.7s
668:	learn: 0.3328507	total: 21.6s	remaining: 10.7s
669:	learn: 0.3328449	total: 21.6s	remaining: 10.6s
670:	learn: 0.3328417	total: 21.7s	remaining: 10.6s
671:	learn: 0.3328382	total: 21.7s	remaining: 10.6s
672:	learn: 0.3328317	total: 21.7s	remaining: 10.6s
673:	learn: 0.3328

814:	learn: 0.3320640	total: 26.3s	remaining: 5.98s
815:	learn: 0.3320596	total: 26.4s	remaining: 5.95s
816:	learn: 0.3320544	total: 26.4s	remaining: 5.91s
817:	learn: 0.3320498	total: 26.4s	remaining: 5.88s
818:	learn: 0.3320453	total: 26.5s	remaining: 5.85s
819:	learn: 0.3320412	total: 26.5s	remaining: 5.82s
820:	learn: 0.3320368	total: 26.5s	remaining: 5.78s
821:	learn: 0.3320328	total: 26.6s	remaining: 5.75s
822:	learn: 0.3320293	total: 26.6s	remaining: 5.72s
823:	learn: 0.3320245	total: 26.6s	remaining: 5.68s
824:	learn: 0.3320192	total: 26.7s	remaining: 5.65s
825:	learn: 0.3320121	total: 26.7s	remaining: 5.62s
826:	learn: 0.3320050	total: 26.7s	remaining: 5.59s
827:	learn: 0.3320000	total: 26.7s	remaining: 5.55s
828:	learn: 0.3319960	total: 26.8s	remaining: 5.52s
829:	learn: 0.3319901	total: 26.8s	remaining: 5.49s
830:	learn: 0.3319847	total: 26.8s	remaining: 5.46s
831:	learn: 0.3319768	total: 26.9s	remaining: 5.43s
832:	learn: 0.3319726	total: 26.9s	remaining: 5.39s
833:	learn: 

975:	learn: 0.3312794	total: 31.4s	remaining: 772ms
976:	learn: 0.3312745	total: 31.4s	remaining: 740ms
977:	learn: 0.3312705	total: 31.5s	remaining: 708ms
978:	learn: 0.3312656	total: 31.5s	remaining: 676ms
979:	learn: 0.3312584	total: 31.5s	remaining: 643ms
980:	learn: 0.3312537	total: 31.6s	remaining: 611ms
981:	learn: 0.3312487	total: 31.6s	remaining: 579ms
982:	learn: 0.3312450	total: 31.6s	remaining: 547ms
983:	learn: 0.3312389	total: 31.7s	remaining: 515ms
984:	learn: 0.3312343	total: 31.7s	remaining: 483ms
985:	learn: 0.3312317	total: 31.7s	remaining: 450ms
986:	learn: 0.3312295	total: 31.8s	remaining: 418ms
987:	learn: 0.3312228	total: 31.8s	remaining: 386ms
988:	learn: 0.3312182	total: 31.8s	remaining: 354ms
989:	learn: 0.3312115	total: 31.8s	remaining: 322ms
990:	learn: 0.3312075	total: 31.9s	remaining: 289ms
991:	learn: 0.3312030	total: 31.9s	remaining: 257ms
992:	learn: 0.3311982	total: 31.9s	remaining: 225ms
993:	learn: 0.3311938	total: 32s	remaining: 193ms
994:	learn: 0.

In [257]:
from sklearn.metrics import accuracy_score
round(accuracy_score(y_test, model.predict(X_test)), 3)

0.894

In [258]:
model.save_model('model', format="cbm")

# Тестируем работу модели при запросе

In [259]:
all_posts['predict'] = model.predict_proba(all_posts.drop('post_id',axis=1))[:, 1]

In [260]:
all_posts

,gender,age,country_Belarus,country_Cyprus,country_Estonia,country_Finland,country_Kazakhstan,country_Latvia,country_Russia,country_Switzerland,...,DistanceTo14thCluster,DistanceTo15thCluster,attraction,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,predict
0,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.274009,3.226359,1,0,0,0,0,0,0,0.098942
1,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.260491,3.178570,0,0,0,0,0,0,0,0.082608
2,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.399966,3.126604,1,0,0,0,0,0,0,0.093992
3,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.584817,3.369332,1,0,0,0,0,0,0,0.084381
4,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.591954,2.823423,1,0,0,0,0,0,0,0.087795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.416083,2.974245,1,0,0,1,0,0,0,0.122910
7019,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.003729,2.832663,1,0,0,1,0,0,0,0.119446
7020,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.761077,2.676703,0,0,0,1,0,0,0,0.112879
7021,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.028654,3.388523,0,0,0,1,0,0,0,0.143404


In [261]:
tuple(all_posts.sort_values('predict', ascending=False)['post_id'].head(5))

(5646, 7171, 7172, 5530, 5112)

# Загружаем модель

In [262]:
import os

def get_model_path(path: str) -> str:
    if os.environ.get("IS_LMS") == "1":  # проверяем где выполняется код в лмс, или локально. Немного магии
        MODEL_PATH = '/workdir/user_input/model'
    else:
        MODEL_PATH = path
    return MODEL_PATH

def load_models():
    model_path = get_model_path("/my/super/path")
    
    model = CatBoostClassifier() 
    
    model.load_model(model_path)
    
    return model

In [218]:
post_test.head(2)

,post_id,text,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,...,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,attraction,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,UK economy facing major risks\n\nThe UK manufa...,8,3.317282,3.626825,3.504385,2.107590,2.673781,3.362377,1.969380,...,3.299252,3.274009,3.226359,1,0,0,0,0,0,0
1,2,Aids and climate top Davos agenda\n\nClimate c...,8,3.175854,3.332851,3.201369,2.080955,2.344407,3.098747,1.947727,...,2.774500,3.260491,3.178570,0,0,0,0,0,0,0


# Выгрузка признаков

In [219]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

df_post_to_sql = post_test

df_post_to_sql.to_sql('tkachenko_a_post_mod_features__lesson_22', if_exists='replace', con=engine, index=False) # записываем таблицу

#df = test
#df_user_to_sql = df_user
#df_post_to_sql = df_post

#df.to_sql('tkachenko_a_features_lesson_22', if_exists='replace', con=engine, index=False) # записываем таблицу
#df_user_to_sql.to_sql('tkachenko_a_user_features__lesson_22', if_exists='replace', con=engine, index=False) # записываем таблицу
#df_post_to_sql.to_sql('tkachenko_a_post_features__lesson_22', if_exists='replace', con=engine, index=False) # записываем таблицу

df = pd.read_sql('SELECT * FROM tkachenko_a_post_mod_features__lesson_22', con=engine) # считываем таблицу


In [220]:
pd.read_sql('SELECT * FROM tkachenko_a_post_mod_features__lesson_22', con=engine) # считываем таблицу

,post_id,text,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,...,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,attraction,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,UK economy facing major risks\n\nThe UK manufa...,8,3.317282,3.626825,3.504386,2.107590,2.673781,3.362377,1.969380,...,3.299252,3.274009,3.226359,1,0,0,0,0,0,0
1,2,Aids and climate top Davos agenda\n\nClimate c...,8,3.175854,3.332851,3.201369,2.080955,2.344407,3.098746,1.947727,...,2.774500,3.260491,3.178570,0,0,0,0,0,0,0
2,3,Asian quake hits European shares\n\nShares in ...,8,3.206180,3.420908,3.228868,2.183235,2.723450,3.286630,2.831849,...,2.775532,3.399967,3.126604,1,0,0,0,0,0,0
3,4,India power shares jump on debut\n\nShares in ...,8,3.616450,2.843541,3.649253,2.582405,3.227174,3.933870,3.203395,...,3.559877,3.584817,3.369332,1,0,0,0,0,0,0
4,5,Lacroix label bought by US firm\n\nLuxury good...,8,2.602314,3.101309,2.821453,1.537728,1.803597,3.012253,2.673844,...,2.418449,2.591953,2.823423,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7314,Although the likeliness of someone focusing on...,0,1.179555,2.352704,2.846943,2.361339,2.511536,3.481241,3.138551,...,3.259588,1.718896,3.117846,1,0,0,1,0,0,0
7019,7315,"OK, I would not normally watch a Farrelly brot...",0,0.879528,2.180947,2.797818,2.564929,2.130065,3.268162,3.195646,...,2.829822,1.416082,2.974245,1,0,0,1,0,0,0
7020,7316,I give this movie 2 stars purely because of it...,0,0.698967,2.086715,2.490488,2.322584,2.111093,3.286735,3.083744,...,3.111119,1.003729,2.832663,1,0,0,1,0,0,0
7021,7317,I cant believe this film was allowed to be mad...,0,1.161841,2.444997,2.193800,2.647159,2.271674,3.361959,3.267647,...,3.014389,1.761077,2.676703,0,0,0,1,0,0,0


In [221]:
import pandas as pd
from sqlalchemy import create_engine


def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

def load_features() -> pd.DataFrame:
    result = batch_load_sql('SELECT * FROM tkachenko_a_features_lesson_22')
    
    return result